# Dublin House Prices by Post Code
## Part 5 - the Maps
We'll use Pascal Bugnion's [excellent gmaps package](https://github.com/pbugnion/gmaps) to map our house data. First, we'll load our data and check it's ok.

In [1]:
import pandas as pd
import pickle
with open('../library/pickle/prices_final_dublin_2016_2015_pickle', 'r') as f:
    houses = pickle.load(f)
houses.head()

,Address,Date,Description,FullMarketPrice,Lat,Lon,Price,Size,VAT,eirKey,eirKeyName
0,"34 Mountpleasant Terrace, Dublin 6, D06 YC58, ...",2016-01-01,Second-Hand Dwelling house /Apartment,No,53.328587,-6.261495,170000.0,,No,D06,Rathmines
1,"2 Brighton Rd, Brighton Hall, Kerrymount, Dubl...",2016-01-04,Second-Hand Dwelling house /Apartment,No,53.258165,-6.174641,1150000.0,,No,D18,Foxrock
2,"24 Woodstown Meadow, Ballycullen, Dublin 16, D...",2016-01-04,Second-Hand Dwelling house /Apartment,No,53.273761,-6.327188,430000.0,,No,D16,Ballinteer
3,"28 Belton Park Gardens, Clontarf, Dublin 9, D0...",2016-01-04,Second-Hand Dwelling house /Apartment,No,53.375715,-6.226541,225000.0,,No,D09,Drumcondra
4,"48A Beaufield Park, Stillorgan, Dublin, A94 XH...",2016-01-04,Second-Hand Dwelling house /Apartment,No,53.290123,-6.203367,345000.0,,No,A94,Blackrock


Next, we'll create a `gmaps` object, `houseMap`, using an authentication key from [Google API](https://console.developers.google.com/apis/).

In [3]:
import gmaps
import json
with open('/Users/anthonymunnelly/Documents/TECH/Google API/api.json', 'r') as f:
    keys = json.load(f)
gmaps.configure(api_key = keys['create_the_map'])

houseMap = gmaps.Map()

`houseMap` will show the locations of the houses sold in Dublin in 2016-to-date.

Having created our map object, we now  need to assemble some data for it. To do this, we'll create a copy of our Lat, Lon and Price columns in the `houses` data frame, make sure there are no null values, and convert each into a list.

As we only need out latitude and longitudinal details first, we create a list of lat-lon tuples and use this list to create a `Heatmap` layer which we add to the `houseMap` object.

In [4]:
forMapping = houses[['Lat', 'Lon', 'Price']].copy()
forMapping = forMapping.dropna(how='any')
forMapping = forMapping.sort_values('Price', ascending = False)
forMapping.head(20)

,Lat,Lon,Price
13326,53.287019,-6.364894,64085057.00
16225,53.346695,-6.384659,18150000.00
14032,53.265648,-6.148350,10500000.00
20759,53.300498,-6.283854,10400000.00
21063,53.261294,-6.119492,9000000.00
14782,53.311221,-6.255564,8725000.00
19175,53.287380,-6.362599,7743017.58
17607,53.259770,-6.113644,7500000.00
1904,53.326149,-6.254584,6500000.00
20279,53.287068,-6.390184,6412500.00


We see the same issue here with regard to the prices, where there are some prices so outlandishly out of adjustment with the majority that they skew the results. So we'll cap house prices at a reasonable five million Euro or under, and map from there.

In [7]:
forMapping = forMapping[forMapping.Price < 10000000]
lat = forMapping['Lat'].tolist()
lon = forMapping['Lon'].tolist()
price = forMapping['Price'].tolist()

count_data = []
[count_data.append((lat[i], lon[i])) for i in range(len(lat))]

count_layer = gmaps.Heatmap(data=count_data, point_radius = 15)
houseMap.add_layer(count_layer)

## House Sales in Dublin, 2016-to-Date

In [9]:
houseMap

### Conclusions
On the second or third zoom, we can see the sharp concentration in area on the extremity of the city - Blanchardstown/Castleknock, Donabate and Dún Laoghaire along the coast, and in the southwest at Saggart, Stepaside and Ballyboden.

# House Prices in Dublin, 2016-to-Date
There is a similar process to create the Prices map. This time, we called `WeightedHeapmap`, which takes a third, Price, value which it uses to color the heatmap.

In [10]:
price_mapping_data = []
[price_mapping_data.append((lat[i], lon[i], price[i])) for i in range(len(lat))]

price_layer = gmaps.WeightedHeatmap(data=price_mapping_data, point_radius=15)
priceMap = gmaps.Map()
priceMap.add_layer(price_layer)

In [11]:
priceMap

### Conclusions
While properties were bought for considerable amounts all across the city and into the rest of Leinster, the angry red colours centered over Ballsbridge and Rathmines indicating the price of real estate in those most des of reses suggest that some things will never really change.